<a href="https://colab.research.google.com/github/sinilga/DataAnalitics/blob/main/Project_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import psycopg2
import psycopg2.extras 
import pandas as pd
import numpy as np
import networkx as nx
import plotly.graph_objects as go
import plotly.express as px

import warnings
warnings.simplefilter('ignore')

pd.set_option('display.precision',2)

# Введение

## Описание пробемы
Клиенты онлайн-школы MasterMind покупают несколько сверх популярных курсов, игнорируя остальные.

## Цель 
Разработать рекомендательную систему, позволяющую предлагать клиентам интересные им курсы, попышая средний чек.

## Задача
Подготовить основу рекомендательной системы в форме таблицы, в которой каждому курсу соответствуют две рекомендации.


# Сбор данных
Получение данных о продажах курсов в разрезе пользователей:

In [ ]:
# Отбираем пользователей и купленные ими курсы. 
# К каждой строке добавляется список идентификаторов других курсов, купленных тем же пользователем

req_course_with_alts = """
select 
    user_id as usr,
    resource_id as course,
    array_remove(array_agg(resource_id) over (partition by user_id),resource_id) as alts
from final.carts as cart
    join final.cart_items ci on cart.id = ci.cart_id
where 
    ci.resource_type = 'Course' and cart.state = 'successful'
"""

In [ ]:
# Разврачиваем список альтернативных курсов из столбца `alts` предыдущего запроса в строки

req_pairs = """
select 
    usr, 
    course, 
    unnest(alts) as alt
from 
    course_with_alts
"""

In [ ]:
# Группируем пары курсов и подсчитываем вес пары - число клиентов, покупавших даную пару курсов

req_course_pairs = f"""
with course_with_alts as (
    {req_course_with_alts}
),
pairs as (
    {req_pairs}
)
select 
	course, 
    alt, 
    count(distinct usr)
from
	pairs
group by
	1,2
order by 
    3 desc;
"""    

In [ ]:
# Дополнительный запрос - для каждого курса получаем оценку его популярности, т.е. число купивших его клиентов.
 
req_course_popularities = """
select 
    resource_id,
    count(distinct user_id) as popularity
from final.carts as cart
    join final.cart_items ci on cart.id = ci.cart_id
where 
    ci.resource_type = 'Course' and cart.state = 'successful'
group by
    resource_id    
"""

In [ ]:
try:
    connection = psycopg2.connect(
        dbname='skillfactory',
        user='skillfactory',
        host='84.201.134.129',
        password='cCkxxLVrDE8EbvjueeMedPKt',
        port='5432'
    )
    cursor = connection.cursor()
    cursor.execute(req_course_pairs)
    records = cursor.fetchall()
    pairs_df = pd.DataFrame(records, columns=['course','alt','weight'])
    cursor.execute(req_course_popularities)
    records = cursor.fetchall()
    courses_df = pd.DataFrame(records, columns=['course','popularity'])
except (Exception, psycopg2.Error) as error:
    print("Error while connecting to PostgreSQL", error)
finally:
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

PostgreSQL connection is closed


Набор `pairs_df` содержит пары курсов: каждому курсу приписаны идентификаторы дополнительных курсов (поле `alt`) и число совместных покупок (`weight`).
Каждая пара в наборе встречается дважды (A -> B, B -> A ).


In [ ]:
display(pairs_df.head())
print(f'Size: {pairs_df.shape}')

,course,alt,weight
0,566,551,797
1,551,566,797
2,515,551,417
3,551,515,417
4,551,489,311


Size: (7978, 3)


Набор `courses_df` содержит перечень курсов и число купивших его клиентов.

In [ ]:
display(courses_df.head())
print(f'Size: {courses_df.shape}')

,course,popularity
0,356,659
1,357,761
2,358,137
3,359,79
4,360,59


Size: (126, 2)


# Предварительное изучение данных

In [ ]:
def show_describe(data, prop, title):
    info = data[prop].describe()

    print(title)
    print(('{:>8s} | '*8).format(*info.index))
    print('-'*87)
    print(('{:8.2f} | '*8).format(*info.values))
    print()

    print(f'Выбросов снизу: {data[data[prop] < info[4] - 1.5*(info[6]-info[4])].shape[0]}')
    print()

    print(f'Выбросов сверху: {data[data[prop] > info[6] + 1.5*(info[6]-info[4])].shape[0]}')
    print()

    fig = px.box(data,x=prop,orientation='h',log_x=True, height=150)
    fig.update_layout(margin = {'l': 5, 'r': 5, 't': 15, 'b': 5})
    fig.update_traces(quartilemethod="linear")
    fig.show()

In [ ]:
show_describe(courses_df, 'popularity', 'Описательная статистика по популярности курсов')


Описательная статистика по популярности курсов
   count |     mean |      std |      min |      25% |      50% |      75% |      max | 
---------------------------------------------------------------------------------------
  126.00 |   558.92 |  1099.05 |     2.00 |    99.25 |   222.50 |   587.00 |  9564.00 | 

Выбросов снизу: 0

Выбросов сверху: 12



Статистические данные по популярности курсов подтверждают наличие проблемы, послужившей основанием проведения данной работы: существует несколько сверхпопулярных курсов, у которых значение данного показателя кратно превышает границу выбросов.

In [ ]:
# общее число пар
print(f'Число пар: {pairs_df.shape[0]/2}')
print('Наиболее популярная пара:')
pairs_df.sort_values(by='weight', ascending=False)[:1]

Число пар: 3989.0
Наиболее популярная пара:


,course,alt,weight
0,566,551,797


In [ ]:
# число воможных пар на курс
display(pairs_df.groupby('course').alt.count().describe())

count    126.00
mean      63.32
std       28.67
min        3.00
25%       44.25
50%       64.00
75%       80.00
max      117.00
Name: alt, dtype: float64

Каждому курсу может соответствовать от 3 до 117 дополнительных курсов. Для включения в таблицу рекомендаций необходимо отобрать два из них.  В качестве критерия отбора будет использоваться показатель веса пары: чем большее число клиентов, купивших курс `course`, купили также курс `alt`, тем сильнее связаны курсы.

Неоднозначность при выборе может создать ситуация, когда у курса есть несколько пар с одинаковым весом.

In [ ]:
# число пар с одинаковым весом
pairs_df.groupby(by=['course','weight'], as_index = False) \
    .agg(alt_counts=pd.NamedAgg(column="alt", aggfunc="nunique")) \
    .sort_values(by=['alt_counts'], ascending=[False])

,course,weight,alt_counts
1727,864,1,39
1718,863,1,35
1575,803,1,34
1736,865,1,34
1871,1102,1,33
...,...,...,...
1209,742,34,1
1207,742,18,1
1206,742,12,1
1204,742,7,1


Чтобы снизить неоднозначность выбора рекомендаций, будем учитывать в качестве дополнительного критерия также популярность второго курса в паре (`alt`). Чем она выше, тем больше шансов, что клиент заинтересуется. 

Вместе с тем, предлагать курсы и без того сверхпопулярные нецелесообразно: во-первых, их клиенты с большой вероятностью купят и без рекомендациий, во-вторых, это противоречило бы цели работы - привлечь внимание клиентов к курсам, которые могут быть им интересны, но почему-то плохо продаются.

Поэтому при выборе будем отдавать предпочтение парам, в которых популярность курса `alt` высока, но не выходит за границу выбросов. Пары со свехпопуляными курсами будем предлагать в крайнем случае, если не найдется других вариантов.

Соединяем сведения о популярности курсов с парами:
- `popularity` - число клиентов, купивших курс `course`
- `alt_popularity` - число клиентов, купивших курс `alt`


In [ ]:
df = pairs_df \
        .merge(courses_df, on='course') \
        .merge(courses_df.rename(columns={'popularity':'alt_popularity','course':'alt_course'}), left_on='alt', right_on='alt_course') 
df = df.drop(columns=['alt_course'])
df.sort_values(by=['course','alt'])

,course,alt,weight,popularity,alt_popularity
4239,356,357,100,659,761
7614,356,360,1,659,59
4933,356,361,17,659,591
3124,356,366,15,659,513
3682,356,367,12,659,297
...,...,...,...,...,...
7538,1200,1104,1,2,122
2031,1201,571,1,5,1054
4365,1201,1125,1,5,810
5912,1201,1144,1,5,184


Проверяем наличие совпадений по весу пары с учетом популярности курса `alt`

In [ ]:
df.groupby(by=['course','weight','alt_popularity'], as_index = False) \
    .agg(alt_counts=pd.NamedAgg(column="alt", aggfunc="nunique")) \
    .sort_values(by=['alt_counts'], ascending=[False])

,course,weight,alt_popularity,alt_counts
757,489,14,339,2
3555,741,2,96,2
3547,741,1,206,2
2581,564,6,206,2
3880,749,1,341,2
...,...,...,...,...
2648,566,3,8,1
2647,566,2,245,1
2646,566,2,137,1
2645,566,2,135,1


По-прежнему существуют пары с одинаковым весом, но теперь для каждого курса их не более двух. Поскольку мы собираемся включать в таблицу по две рекомендации, неоднозначности это не создаст.

# Подготовка рекомендательной системы

## Выбор пар
Проранжируем пары по числу клиентов, купивших курс `alt` совместо с курсом `course`, и по популярности курса `alt`. 
И включим в перечень рекомендации пары с рангами 1 и 2.

In [ ]:
# Для каждого курса ранжируем пары по относительному весу, а атакже по популярности дополнительного курса, отсекая выбросы 
top_outlier = courses_df.popularity.quantile([0.25,0.75]).agg(lambda x: x.max() + 1.5*(x.max()-x.min()))

def sort_key(col):
    return col.apply(lambda x: x if x < top_outlier else top_outlier - x)

df = df.sort_values(by=['alt_popularity'],ascending=[False], key=sort_key)
df = df.sort_values(by=['course','weight'],ascending=[True,False])
df['priority'] = df.groupby('course')['course'].transform(lambda x: range(1,len(x)+1))

# Оставляем для включения в итоговую таблицу первые две рекомендации для каждого курса
advice_df = df[df.priority < 3].copy()

In [ ]:
# вес пар, попавших в рекомендации
px.box(advice_df,'weight', log_x=True, height=200)

Некоторые из попавших в рекомендации пар имеют малый вес (1,2,...). 

Это не обязательно свидетельствует о слабой связи курсов: возможно сам курс `course` является новым и вследствие этого был приобретен небольшим числом клиентов (вес пары по определению не может превышать популярности приобретаемого курса).

Поэтому при оценке силы связи между курсами целесообразно рассматривать относительный вес пары - долю клиентов купивших пару `course -> alt` среди купивших курс `course`.

In [ ]:
# расчитываем относительный вес пары
advice_df['weight_rel'] = round(100*advice_df['weight']/advice_df['popularity'],2)
px.box(advice_df,'weight_rel', log_x=True, height=200)

In [ ]:
px.box(advice_df.drop_duplicates('alt'),'alt_popularity', log_x=True, height=200)

В список рекомендаций вошли пары с достаточно низким относительным весом (от 2,8%). Это может свидетельствовать об отсутствии тесной связи между составляющими пару курсами. В таком случае целесообразно предложить клиенту какой-либо иной вариант.

Вместе с тем следует учитывать, что:

1) поскольку мы выбирали для рекомендаций пары максимального веса, то любой другой вариант скорее всего будет еще менее тесно связан с приобретаемым курсом;

2) часто даже пара с низким весом ведет к популярному курсу и вероятность, что клиент им заинтересуется высока.

Таким образом, замену рекомендаций целесообразно осуществлять лишь в исключительныых случаях, когда и вес пары, и популярность рекомендуемого курса являются низкими.

Можно предложиь следующий вариант коррекции рекомендаций:
отобрать пары с низким весом и ведущие к малопопуярным курсам, и заменить в них курс `alt` на один из курсов, рекомендованных для него (т.е. следующих за ним в цепочке рекомендаций) и имеющих популярность, превышающую установленный порог.

Поскольку выбросов снизу нет ни по показателю относительного веса пары, ни по показателю популярности курсов, в качестве нижней границы в обоих случаях выберем первый квартиль.

In [ ]:
# пары - кандидаты а замену

min_weight = advice_df.weight_rel.quantile(0.25)
min_popularity = advice_df.drop_duplicates('alt').alt_popularity.quantile(0.25)

advice_df[(advice_df.weight_rel < min_weight) & (advice_df.alt_popularity < min_popularity)].sort_values("weight_rel")


,course,alt,weight,popularity,alt_popularity,priority,weight_rel
7491,365,1141,3,56,190,2,5.36
7252,361,1138,40,591,245,2,6.77
5281,1125,912,55,810,213,2,6.79
5763,1160,563,4,55,166,2,7.27


In [ ]:
min_weight = advice_df.weight_rel.quantile(0.25)
min_popularity = advice_df.drop_duplicates('alt').alt_popularity.quantile(0.25)

# функция выбра курса для замены малопопулярного курса
# для каждой пары, выбранной для замены рекомендуемого кураса
# обходим граф рекомендаций в ширину начиная от вершины, соответствующей курсу alt
# обход завершается, когда найден курс с популарностью выше min_popularity или просмотрены все вершины
def improove(row,data):
    if row.weight_rel < min_weight and row.alt_popularity < min_popularity:
        other_alt = data[(data.course == row.course)&(data.alt != row.alt)].alt
        done = (data.course == row.course)  # маска просмотренных вершин
        tmp_df = pd.DataFrame([row])
        cur_max = row.alt_popularity
        while (cur_max < min_popularity) and (not done.all()):
            tmp_df = tmp_df.merge(data[~done], left_on='alt', right_on='course')
            done = done | data.course.isin(tmp_df.alt_x)
            tmp_df = tmp_df[['course_x','alt_y','alt_popularity_y']]
            tmp_df.columns=['course','alt','alt_popularity']
            cur_max = tmp_df[~tmp_df.alt.isin(other_alt)].alt_popularity.max()
        if cur_max >= min_popularity:
            return tmp_df[(~tmp_df.alt.isin(other_alt)) & (tmp_df.alt_popularity == cur_max)].alt.min()
    return row.alt

r = advice_df.apply(lambda row: improove(row,advice_df), axis=1)
advice_df['new_alt'] = r.apply(lambda x: int(x))
advice_df[advice_df.new_alt != advice_df.alt][['course','alt','new_alt']]


,course,alt,new_alt
7252,361,1138,361
7491,365,1141,513
5281,1125,912,1125
5763,1160,563,363


In [ ]:
# корректируем weight и alt_popularity в соответствии с новыми рекомендациями
def weight_correction(row):
    row.alt = row.new_alt
    row.alt_popularity = courses_df[courses_df.course==row.new_alt].popularity.min()    
    x = df[(df.course==row.course)&(df.alt==row.new_alt)].weight.min()    
    if not np.isnan(x):
        row.weight = -1 if np.isnan(x) else x
        row.weight_rel = 100*row.weight/row.popularity
    else:    
        row.weight = -1
        row.weight_rel = -1
    return row

advice_df = advice_df.apply(weight_correction, axis=1)

## Визуализация рекомендательной системы

### Вспомогательные функции для отрисовки графа

In [ ]:
node_size = 25                  
node_color = '#6fa8dc'
node_border = '#3d85c6'
q1,q2,q3 = list(advice_df['weight_rel'].quantile([0.25,0.5,0.75]))

# отрисовка ребер графа
def draw_edges(G,pos):
    edge_colors = ['red','#A0A0A0','#606060','#404040','#202020','black']
    edge_weight = [1, 1, 1, 1.5, 2, 2.5]
    q1,q2,q3 = np.quantile(list(nx.get_edge_attributes(G,'weight_rel').values()),[0.25,0.5,0.75])
    irq = q3-q1
    edge_lims = [q1-1.5*irq, q1, q2, q3, q3+1.5*irq]

    # окраска ребер в зависимости от веса пары
    def edge_class(val):
        for k, lim in enumerate(edge_lims):
            if val < lim:
                return k
        return len(edge_lims)

    edge_x = [ [] for i in range(len(edge_lims) + 1) ]
    edge_y = [ [] for i in range(len(edge_lims) + 1) ]
    hovertext = [ [] for i in range(len(edge_lims) + 1) ]
    edge_traces = []
    edge_lab = []

    for edge, attr in G.edges().items():
        p0, p1 = edge[0], edge[1]
        x0, y0 = pos[p0][0], pos[p0][1]
        x1, y1 = pos[p1][0], pos[p1][1]
        cls = edge_class(attr['weight_rel'])
        edge_x[cls].extend([x0, x1, None])
        edge_y[cls].extend([y0, y1, None])
        edge_lab.append(
            go.Scatter(
                x = [(x0+x1)/2,None], y = [(y0+y1)/2,None],
                mode = 'markers',
                text=f'{attr["weight_rel"]}',
                hovertext=f'Пара: {edge[0]} -> {edge[1]}:<br>Отн. вес: {attr["weight_rel"]:.0f}%<br>Абс. вес: {attr["weight"]:.0f}',
                hoverinfo='text',
                hoveron='fills+points',
                marker = {'color':'white','size':10},
                opacity=0,
                showlegend=False,
            )
        )
        hovertext[cls].append(f'{edge[0]}->{edge[1]}: weight {attr["weight_rel"]}')

    for idx in range(len(edge_lims) + 1):
        col,width = edge_colors[idx], edge_weight[idx]
        name = ''
        if idx == 0:
            name = f'< {edge_lims[0]:.0f}%'
        elif idx == len(edge_lims):    
            name = f'> {edge_lims[idx-1]:.0f}%'
        else:
            name = f'{edge_lims[idx-1]:.0f}% - {edge_lims[idx]:.0f}%'

        edge_traces.append(go.Scatter(
            x = edge_x[idx], y = edge_y[idx], # x = [x0,x1], y = [y0,y1],
            line = {'width':width,'color':col},
            marker = {'angle':0,'angleref':'previous','standoff':int(0.5+node_size/2),'symbol':53,'size':10},
            name=name,
            fill='toself',
            fillcolor = col,
            hoverinfo = 'text',
            hoveron='fills+points',
            mode = 'lines+markers')
        )

    return edge_traces + edge_lab
    
# отрисовка вершин
def draw_nodes(G,pos):
    node_x = []
    node_y = []
    names=[]
    for node, attr in G.nodes().items():
        p = pos[node]
        node_x.append(p[0])
        node_y.append(p[1])
        names.append(f'Курс: id={node}<br><br>популярность:{ attr["popularity"]}<br>связи: { list(nx.neighbors(G,node))}')
    
    node_trace = go.Scatter(
        x = node_x, y = node_y,
        mode = 'markers',
        text=node,
        hoverinfo='text',
        hovertext = names,
        hoveron='fills+points',
        marker = {'color':node_color,'size':node_size,'line':dict(color = node_border,width=1)},
        showlegend=False,
    )
    
    return node_trace

# метки вершин
def node_labels(G,pos):
    info=[]
    for node,p in pos.items():
        info.append(
            go.layout.Annotation(
                text = node,
                x = p[0],
                y = p[1],
                xanchor='center',
                yanchor='middle',
                font = dict(size=8 if node >= 1000 else 10, color='black'),
                showarrow=False
            )
        )
    return info   

# отрисовка графа
def draw_graph(G):
    pos = nx.nx_pydot.graphviz_layout(G)
    edge_trace = draw_edges(G,pos)
    node_trace = draw_nodes(G,pos)
    title = {
        'text':'Система рекомендаций',
        'pad': dict(t=10,b=0,l=0,r=0),
        'font':dict(size=14),
        'yanchor': 'top',
        'xanchor': 'left',
        'x':0.01,
        'y':1,
    }
    fig = go.Figure(data=edge_trace + [node_trace],
            layout=go.Layout(
                width = 1000, height = 800, 
                margin = {'l': 0, 'r': 0, 't': 30, 'b': 0},
                title=title,
                showlegend = True,
                xaxis = {'showgrid': False, 'zeroline': False, 'showticklabels': False},
                yaxis = {'showgrid': False, 'zeroline': False, 'showticklabels': False},
                annotations = node_labels(G,pos), 
                legend = dict(
                    orientation='h', 
                    x=0.01, y=1, xanchor='left', yanchor='top', 
                    title={'text':'Вес пары:'},
                    itemsizing = 'trace',
                    itemclick = 'toggleothers'
                )
            )
    )
    fig.show()

### Вывод графа

In [ ]:
G = nx.from_pandas_edgelist(advice_df,source='course',target='alt',edge_attr=['weight_rel','weight'],create_using=nx.DiGraph())
nx.set_node_attributes(G,courses_df.groupby('course').min().popularity.to_dict(),'popularity')
q1,q2,q3 = list(advice_df['weight_rel'].quantile([0.25,0.5,0.75]))
draw_graph(G)

## Итоговая таблица рекомендаций

In [ ]:
# Собираем таблицу рекомендаций
advice_pivot = advice_df.pivot_table(index='course', columns='priority',values=['alt'],aggfunc=lambda x:int(x))
advice_pivot.reset_index(inplace=True)
advice_pivot.columns = ['Курс','Рекомендация 1','Рекомендация 2']
advice_pivot

,Курс,Рекомендация 1,Рекомендация 2
0,356.0,571,357
1,357.0,571,356
2,358.0,570,752
3,359.0,570,358
4,360.0,745,516
...,...,...,...
121,1188.0,1141,513
122,1198.0,571,912
123,1199.0,523,570
124,1200.0,1104,866


# Заключение

Построенная таблица включает по две рекомендации для каждого из 126 курсов, которые приобретались клиентами. 

Представление рекомендаций в форме графа наглядно показывает, что сушествует несколько центров приитяжения - групп вершин, связанных исходящими ребрами только друг с другом. Клиент следуя нашим рекомендациям быстро попадет в один их таких центров и уже не сможет из них выйти.





In [ ]:
print('Центры притяжения')
i = 1
for c in nx.attracting_components(G):
    lst = G.subgraph(c).nodes()
    print(f'{i}. {len(lst)} вершин : {lst}')
    i += 1

Центры притяжения
1. 4 вершин : [357.0, 571.0, 356.0, 1125.0]
2. 3 вершин : [752.0, 570.0, 507.0]
3. 3 вершин : [745.0, 516.0, 553.0]
4. 5 вершин : [515.0, 551.0, 489.0, 566.0, 794.0]
5. 3 вершин : [562.0, 363.0, 511.0]
6. 4 вершин : [504.0, 569.0, 840.0, 572.0]


Чтобы избежать таких зацикливаний целесообразно дополнить данную таблицу другими рекомендацями, основанными не на групповом выборе клиентов, а, к примеру, на тематической близости курсов или разумной (по мнению преподавателей школы) последовательности их изучения. 

Также можно искусственно ввести дополнительные курсы общеразвивающего характера, интересные разным группам клиентов, и через них связать отдельные замкнутые группы курсов. 